In [ ]:
!pip install pyspark

In [ ]:
!pip install graphframes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   164  100   164    0     0    915      0 --:--:-- --:--:-- --:--:--   916
100   146  100   146    0     0    260      0 --:--:-- --:--:-- --:--:--   260
curl: (23) Failure writing output to destination


In [ ]:
from tqdm import tqdm
import pandas as pd
import glob
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from google.colab import drive
import os
from graphframes import GraphFrame

In [ ]:
import zipfile
import os
from google.colab import files
uploaded = files.upload()
# Unzip
with zipfile.ZipFile("Lab6.zip", 'r') as zip_ref:
    zip_ref.extractall()

Saving Lab6.zip to Lab6 (1).zip


In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12").getOrCreate()

In [ ]:
station_path = "Lab6/station_data.csv"
station_data_df = spark.read.csv(station_path, header=True, inferSchema=True)

station_data_df.printSchema()


root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)



In [ ]:
trip_path = "Lab6/trip_data.csv"
trip_data_df = spark.read.csv(trip_path, header=True, inferSchema=True)

trip_data_df.printSchema()

root
 |-- Trip ID: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Start Station: string (nullable = true)
 |-- Start Terminal: integer (nullable = true)
 |-- End Date: string (nullable = true)
 |-- End Station: string (nullable = true)
 |-- End Terminal: integer (nullable = true)
 |-- Bike #: integer (nullable = true)
 |-- Subscriber Type: string (nullable = true)
 |-- Zip Code: integer (nullable = true)



In [ ]:
station_data_df = station_data_df.withColumnRenamed('name', 'id')

In [ ]:
trip_data_df = trip_data_df.withColumnRenamed('Start Station', 'src').withColumnRenamed('End Station', 'dst')

In [ ]:
graph = GraphFrame(station_data_df, trip_data_df)

# vertex table == station data
# edge table == trip data

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [ ]:
#Return the number of trips made between each source and destination, sorted in descending order (based on the number of trips).
trip_counts = graph.edges.groupBy("src", "dst").count().orderBy("count", ascending=False)

trip_counts.show()

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|San Francisco Cal...|    3|
|San Francisco Cal...|  Powell Street BART|    2|
|     2nd at Townsend|   Market at Sansome|    2|
|     Spear at Folsom|     2nd at Townsend|    2|
|   Market at Sansome|Broadway St at Ba...|    2|
|    Davis at Jackson|Embarcadero at Sa...|    2|
|San Francisco Cal...|   2nd at South Park|    2|
|   Steuart at Market|San Francisco Cal...|    2|
|       Market at 4th|San Francisco Cal...|    2|
|Embarcadero at Fo...|Embarcadero at Sa...|    2|
|Mountain View Cal...|Rengstorff Avenue...|    1|
|    Davis at Jackson|Temporary Transba...|    1|
|       5th at Howard|     Townsend at 7th|    1|
|   Market at Sansome|South Van Ness at...|    1|
|     Beale at Market|Temporary Transba...|    1|
|     2nd at Townsend|Powell at Post (U...|    1|


In [ ]:
# Return the number of trips that start or end at the station "Townsend at 7th", sorted in descending order (based on the number of trips).
graph.edges.filter('src = "Townsend at 7th" or dst == "Townsend at 7th"').groupBy("src", "dst").count().orderBy("count", ascending=False)
trip_counts.show()

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|San Francisco Cal...|    3|
|San Francisco Cal...|  Powell Street BART|    2|
|     2nd at Townsend|   Market at Sansome|    2|
|     Spear at Folsom|     2nd at Townsend|    2|
|   Market at Sansome|Broadway St at Ba...|    2|
|    Davis at Jackson|Embarcadero at Sa...|    2|
|San Francisco Cal...|   2nd at South Park|    2|
|   Steuart at Market|San Francisco Cal...|    2|
|       Market at 4th|San Francisco Cal...|    2|
|Embarcadero at Fo...|Embarcadero at Sa...|    2|
|Mountain View Cal...|Rengstorff Avenue...|    1|
|    Davis at Jackson|Temporary Transba...|    1|
|       5th at Howard|     Townsend at 7th|    1|
|   Market at Sansome|South Van Ness at...|    1|
|     Beale at Market|Temporary Transba...|    1|
|     2nd at Townsend|Powell at Post (U...|    1|


In [ ]:
# Return the vertices that have never been a destination for a trip starting from "Spear at Folsom".
# 1. Get the destinations of trips starting from "Spear at Folsom"
destinations_from_spear = trip_data_df.filter(trip_data_df.src == "Spear at Folsom").select("dst").distinct()

# 2. Get all station IDs (vertices)
all_stations = station_data_df.select("id")

# 3. Subtract the destinations to find stations never reached from "Spear at Folsom"
never_reached = all_stations.subtract(destinations_from_spear)

# 4. Join to get full station info (optional)
never_reached_vertices = station_data_df.join(never_reached, station_data_df.id == never_reached.id)

never_reached_vertices.show()


+----------+--------------------+---------+-----------+---------+------------+------------+--------------------+
|station_id|                  id|      lat|       long|dockcount|    landmark|installation|                  id|
+----------+--------------------+---------+-----------+---------+------------+------------+--------------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|    San Jose|    8/6/2013|San Jose Diridon ...|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|    San Jose|    8/5/2013|San Jose Civic Ce...|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|    San Jose|    8/6/2013|Santa Clara at Al...|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|    San Jose|    8/5/2013|    Adobe on Almaden|
|         6|    San Pedro Square|37.336721|-121.894074|       15|    San Jose|    8/7/2013|    San Pedro Square|
|         7|Paseo de San Antonio|37.333798|-121.886943|       15|    San Jose|    8/7/2013|Paseo

In [ ]:
#8) Return the station with the maximum number of incoming trips.
graph.inDegrees.orderBy("inDegree", ascending=False).limit(1).show()

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|       9|
+--------------------+--------+



In [ ]:
#9) Return the trip with the longest duration.
graph.edges.orderBy('Duration', ascending=False).limit(1).show()

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913386|    1808|8/31/2015 20:23|Embarcadero at Br...|            54|8/31/2015 20:53|Harry Bridges Pla...|          50|   524|     Subscriber|   94105|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+



In [ ]:
# 10)Create a subgraph that contains only the trips that start or end at "Townsend at 7th".
graph_townsend = GraphFrame(station_data_df, trip_data_df.filter((trip_data_df['src'] == 'Townsend at 7th') | (trip_data_df['dst'] == 'Townsend at 7th')))

# vertices == stations
# edges == trips

In [ ]:
# 11) Return all paths that form a "triangle" pattern between three stations.
graph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)").show(5)

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|{49, Spear at Fol...|{69, San Francisc...|{61, 2nd at Towns...|
|{49, Spear at Fol...|{69, San Francisc...|{65, Townsend at ...|
|{49, Spear at Fol...|{69, San Francisc...|{64, 2nd at South...|
|{49, Spear at Fol...|{69, San Francisc...|{64, 2nd at South...|
|{49, Spear at Fol...|{69, San Francisc...|{65, Townsend at ...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# 12) Return all paths that pass through three vertices and start from "Townsend at 7th".
graph_townsend.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(d)").filter("a.id = 'Townsend at 7th'").show(5)

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------------------+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|                   d|
+--------------------+--------------------+--------------------+--------------------+
|{65, Townsend at ...|{49, Spear at Fol...|{65, Townsend at ...|{49, Spear at Fol...|
|{65, Townsend at ...|{49, Spear at Fol...|{65, Townsend at ...|{50, Harry Bridge...|
+--------------------+--------------------+--------------------+--------------------+

